<a href="https://colab.research.google.com/github/ldmontibeller/mestrado/blob/main/Logic_gates_tensorflow_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Portas lógicas com Redes Neurais
Esse caderno visa o entendimento de soluções de referência como TensorFlow e Keras aplicando em simples exemplos de portas lógicas.

##Introdução

### Importação das bibliotecas
Keras é uma biblioteca de alto nível para TensorFlow, que já vem dentro do mesmo.

In [1]:
import numpy as np
import tensorflow as tf

### Montagem do drive

In [127]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Porta lógica AND

### Preparando um dataset de um arquivo txt

In [213]:
data = np.genfromtxt('/content/drive/MyDrive/RC18EE---Intro-to-Deep-Learning-master/L03_perceptron/code/data/perceptron_and.txt', delimiter= '\t')
data

array([[0., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 1., 1.]])

### Sintaxe de arrays em numpy

* : → Seleciona todas as linhas.
* :2 → Seleciona as duas primeiras colunas (índices 0 e 1).
* 2 → Seleciona a terceira coluna (índice 2).

In [214]:
X, y = data[:,:2], data[:,2]
X, y

(array([[0., 0.],
        [0., 1.],
        [1., 0.],
        [1., 1.]]),
 array([0., 0., 0., 1.]))

### Embaralhar dados
Embaralhar dados torna o aprendizado do perceptron mais eficiente. [Explicação aqui](https://https://datascience.stackexchange.com/questions/24511/why-should-the-data-be-shuffled-for-machine-learning-tasks)

In [215]:
def shuffle_in_unison(a, b):
  """Shuffles two arrays in unison using the same permutation.

  Args:
    a: The first numpy array.
    b: The second numpy array.

  Returns:
    A tuple containing the shuffled arrays.
  """
  assert len(a) == len(b)  # Make sure arrays have the same length
  p = np.random.permutation(len(a))  # Create a permutation of indices
  return a[p], b[p]  # Apply the permutation to both arrays

X, y = shuffle_in_unison(X, y)
X, y

(array([[0., 0.],
        [1., 1.],
        [1., 0.],
        [0., 1.]]),
 array([0., 1., 0., 0.]))

### Normalização e separação de dados para validação cruzada
Normalmente um conjunto de dados precisa ser normalizado e ter uma parte sua separada para utilizar um método de validação cruzada. A normalização faz com que os pesos sejam menores e mais fáceis de calcular, mais sobre [aqui](https://www.codecademy.com/article/normalization). Devido a natureza introdutória deste caderno não iremos abordar validação cruzada.

OBS: Por algum motivo normalizar no caso da porta lógica está prejudicando o resultado. Ou o resultado também está saindo normalizado, o que não entendo completamente.

In [216]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# scaler.fit(X)

# X = scaler.transform(X)

# #A média precisa se aproximar de zero e o desvio padrão de 1

# print('X mean', np.mean(X))
# print('X standard deviation', np.std(X))

X


array([[0., 0.],
       [1., 1.],
       [1., 0.],
       [0., 1.]])

### Criando um modelo para rede neural

In [217]:
#Formato de entrada da rede
X_input = tf.keras.Input(shape=(X.shape[1],))

#Formato da única camada de saída
y_pred= tf.keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer="zeros", use_bias='yes')(X_input)

#Criando o modelo com os formatos de entrada e saída
model = tf.keras.Model(inputs=X_input, outputs=y_pred, name='AND_model')

#Outra maneira de criar
# model = tf.keras.Sequential(
#     [
#         tf.keras.layers.Dense(units=1, input_shape=(2,)),
#         tf.keras.layers.Activation(tf.keras.activations.sigmoid)
#     ]
# )


#model.compile(loss= 'binary_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.SGD(learning_rate=0.5))
model.compile(loss= 'binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.5))


model.summary()


Model: "AND_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_45 (InputLayer)     │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_33 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

In [218]:
model.fit(X, y, epochs=100, batch_size=1, verbose=0)

In [219]:
predicted_y = model.predict(np.array([[0,0],[0,1],[1,0],[1,1]]))
predicted_y

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step


array([[0.0011442 ],
       [0.08637983],
       [0.08665108],
       [0.88675505]], dtype=float32)

## Porta lógica XOR e redes multicamadas

Primeiramente vamos criar e embaralhar um dataset para a porta XOR

In [223]:
XOR_data = np.array([[0,0,0],[0,1,1],[1,0,1],[1,1,0]], dtype=np.float64)
X_xor, y_xor = XOR_data[:,:2], XOR_data[:,2]
X_xor, y_xor = shuffle_in_unison(X_xor, y_xor)
X_xor, y_xor

(array([[0., 0.],
        [1., 1.],
        [0., 1.],
        [1., 0.]]),
 array([0., 0., 1., 1.]))

In [224]:
#Formato de entrada da rede
X_input = tf.keras.Input(shape=(X_xor.shape[1],))

#Formato da única camada de saída
y_pred= tf.keras.layers.Dense(units=1, activation='sigmoid', kernel_initializer="zeros", use_bias='yes')(X_input)

#Criando o modelo com os formatos de entrada e saída
model2 = tf.keras.Model(inputs=X_input, outputs=y_pred, name='XOR_model')

#model.compile(loss= 'binary_crossentropy', metrics=['accuracy'], optimizer=tf.keras.optimizers.SGD(learning_rate=0.5))
model2.compile(loss= 'binary_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.5))

model2.summary()

Model: "XOR_model"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_46 (InputLayer)     │ (None, 2)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_34 (Dense)                │ (None, 1)              │             3 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3 (12.00 B)

 Trainable params: 3 (12.00 B)

 Non-trainable params: 0 (0.00 B)

# Dúvidas
1. Normalizar melhora o resultado, por que? Faz com que os pesos sejam menores e mais fáceis de calcular.
2. Mudar ativação para linear, piora muito. Por que? Porque o regressor você quer encontrar um valor predito e não um rótulo, logo não tem um condicional e por isso que a ativação linear é uma identidade, um filtro passa-tudo.
3. Rodar por mais épocas melhora muito o classificador, mas por que se no exemplo do perceptron feito a mão precisamos de somente 5 épocas? É porque o perceptron feito a mão conseguimos modelar uma não linearidade, a função de ativação de um degrau binário, já para ser computável por diferenciação a ativação sigmoide é a mais adequada mas ela precisa de mais gerações para se aproximar de um degrau binário. A taxa de aprendizagem também apresenta um papel fundamental, quanto maior, menos épocas para convergir, no entanto maior o risco de cair em um mínimo local (não é o nosso caso pois o conjunto de dados e características é muito pequeno), outro risco também é se isntabilizar e não convergir. Mais sobre o learning rate [aqui](https://www.jeremyjordan.me/nn-learning-rate/)


# Anotações

* Classificador para categorizar de 0 a 1 (ou nos rótulos disponíveis)
* Regressor linear que classificar de 0 a Infinito, "estou medindo o quanto eu erro"
* Regressão logística (é um classificador) otimiza os casos próximos da reta de decisão (casos duvidoso) e não se importa para dados muito distantes da reta. É desenvolvido através de binary crossentrophy (neg log-likehood) e ativação de sigmoide logística.